In [ ]:
import sys
sys.path.insert(0, '..')
module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.aliproduct_blip_model import ALIPRODUCT_BLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode


In [2]:


'''Edit this to specify checkpoints of models you want to test 
defined as key value pair of {"checpoint name/identifier:"full path to checkpoint"}'''

checkpoints = {"base":"https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_retrieval_coco.pth",
"epoch_1":"/home/ubuntu/Desktop/Retrieval_aliproduct2_blip_large/save_checkpoint_0.pth",
"epoch_2":"/home/ubuntu/Desktop/Retrieval_aliproduct2_blip_large/save_checkpoint_1.pth",
"epoch_3":"/home/ubuntu/Desktop/Retrieval_aliproduct2_blip_large/save_checkpoint_2.pth",
"epoch_4":"/home/ubuntu/Desktop/Retrieval_aliproduct2_blip_large/save_checkpoint_3.pth",
"epoch_5":"/home/ubuntu/Desktop/Retrieval_aliproduct2_blip_large/save_checkpoint_4.pth",
"epoch_6":"/home/ubuntu/Desktop/Retrieval_aliproduct2_blip_large/save_checkpoint_5.pth",
"epoch_7":"/home/ubuntu/Desktop/Retrieval_aliproduct2_blip_large/save_checkpoint_6.pth",
"epoch_8":"/home/ubuntu/Desktop/Retrieval_aliproduct2_blip_large/save_checkpoint_7.pth"}

#Edit this to specify the augmented/original captions you want to test the model checkpoints on 
caption_to_test =  ["caption"]


#path to validation data after preporcessing into csv file format 
dataframe_path = "/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/data/val_data_prompt_clean.csv"

In [ ]:
image_size = 384
preprocess = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ]) 


In [ ]:
def test_checkpoint(caption_col,clip_model,dataframe_path):
    test_loader,df = prepare_data(dataframe_path,
    CONFIG.test_image_data_dir,CONFIG.test_image_data_folder
    ,CONFIG.test_image_col,caption_col,100,preprocess,CONFIG.global_random_state,test=True,tokenize=False)
    trainer = Trainer(gpus=1)
    pred = trainer.predict(clip_model,test_loader)
    full_pred = tuple(map(torch.concat, zip(*pred)))
    image_embed,text_embed = full_pred
    # print(image_embed.size())
    # print(text_embed.size())
    faiss_index = faiss.IndexFlatIP(256)
    faiss_index.add(image_embed.numpy().astype(np.float32))
    top5_k_e,top5_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),5)
    top10_k_e,top10_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),10)
    top5_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top5_k_y_pred)])
    print(caption_col,"num correct pred",sum(top5_preds))
    top10_preds = np.array([1 if y_true in y_pred else 0 for y_true,y_pred in zip(df.index.values,top10_k_y_pred)])
    print(caption_col,"num correct pred",sum(top10_preds))
    top5_acc = top5_preds[top5_preds ==1].shape[0] / top5_preds.shape[0]
    top10_acc = top10_preds[top10_preds ==1].shape[0] / top10_preds.shape[0]
    return top5_acc,top10_acc

In [ ]:


top5_acc_preds = []
top10_acc_preds = []
captions_tested = []
for cap in caption_to_test:
    for checkpoint in checkpoints.keys():
        clip_model =ALIPRODUCT_BLIP(checkpoints[checkpoint],image_size,vit="large")
        top5_acc,top10_acc = test_checkpoint(cap,clip_model,dataframe_path)
        top5_acc_preds.append(top5_acc)
        top10_acc_preds.append(top10_acc)
        captions_tested.append(cap)


In [ ]:
pred_df = {"model_checkpoint":checkpoints.keys(),"top_5":top5_acc_preds,"top_10":top10_acc_preds}

In [ ]:
pred_df = pd.DataFrame(pred_df)

In [ ]:
pred_df